In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)

df = catalog.load('raw/openaire/researchproduct#parquet')

                    INFO     Loading data from raw/openaire/researchproduct#parquet             ]8;id=939333;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=127781;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185373 entries, 0 to 185372
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   author              185305 non-null  object
 1   openAccessColor     53613 non-null   object
 2   publiclyFunded      121560 non-null  object
 3   type                185373 non-null  object
 4   language            185373 non-null  object
 5   subjects            183336 non-null  object
 6   mainTitle           185373 non-null  object
 7   description         183327 non-null  object
 8   publicationDate     185361 non-null  object
 9   publisher           76578 non-null   object
 10  source              68904 non-null   object
 11  format              170917 non-null  object
 12  bestAccessRight     99741 non-null   object
 13  container           48356 non-null   object
 14  id                  185373 non-null  object
 15  originalId          185373 non-null  object
 16  in

In [3]:
df.columns


Index(['author', 'openAccessColor', 'publiclyFunded', 'type', 'language',
       'subjects', 'mainTitle', 'description', 'publicationDate', 'publisher',
       'source', 'format', 'bestAccessRight', 'container', 'id', 'originalId',
       'indicators', 'instance', 'isGreen', 'isInDiamondJournal',
       'contributor', 'pid', 'country', 'contactPerson', 'embargoEndDate',
       'coverage', 'version', 'size', 'filter_applied'],
      dtype='object')

## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [ ]:
def land_openaire_rel_researchproduct_instance(df: pd.DataFrame)-> pd.DataFrame:

    expected_columns = [
        'id',
        'instance',
        'url',
    ]

    # Agregar columnas faltantes con NaN
    for col in expected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[expected_columns].copy()
    df = df.convert_dtypes()
    df.reset_index(drop=True, inplace=True)

    # url
    df_researchproduct2instance = df.explode('instance').reset_index(drop=True)
    df_researchproduct2instance = df_researchproduct2instance[['id','instance']]
    df_instance = pd.json_normalize(df_researchproduct2instance['instance']).reset_index(drop=True)
    df_researchproduct2instance = pd.concat([df_researchproduct2instance.drop(columns=['instance']), df_instance], axis=1)

    df_researchproduct2url = df_researchproduct2instance[['id','url']]
    df_researchproduct2url = df_researchproduct2url.explode('url')

    df_researchproduct2alternateIdentifier = df_researchproduct2instance[['id','alternateIdentifier']]
    df_researchproduct2alternateIdentifier = df_researchproduct2alternateIdentifier.explode('alternateIdentifier').reset_index(drop=True)
    df_alternateIdentifier = pd.json_normalize(df_researchproduct2alternateIdentifier['alternateIdentifier']).reset_index(drop=True)
    df_researchproduct2alternateIdentifier = pd.concat((df_researchproduct2alternateIdentifier.drop(columns=['alternateIdentifier']), df_alternateIdentifier), axis=1)

    df_researchproduct2url['load_datetime'] = date.today()

    df_researchproduct2instance.drop(columns=['alternateIdentifier','articleProcessingCharge','url','pid'], inplace=True)
    df_researchproduct2url.dropna(inplace=True)
    df_researchproduct2alternateIdentifier.dropna(inplace=True)
    
    df_researchproduct2instance['load_datetime'] = date.today()
    df_researchproduct2url['load_datetime'] = date.today()
    df_researchproduct2alternateIdentifier['load_datetime'] = date.today()

    return df_researchproduct2instance, df_researchproduct2url, df_researchproduct2alternateIdentifier


In [ ]:
df_researchproduct2instance, df_researchproduct2url, df_researchproduct2alternateIdentifier = land_openaire_rel_researchproduct_instance(df)

In [22]:
df_researchproduct2instance

,id,license,publicationDate,refereed,type,articleProcessingCharge.amount,articleProcessingCharge.currency,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,CC BY,2023-12-18,nonPeerReviewed,Article,NaN,NaN,2025-02-28
1,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,None,2014-01-01,peerReviewed,Conference object,NaN,NaN,2025-02-28
2,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,None,2013-01-01,peerReviewed,Conference object,NaN,NaN,2025-02-28
3,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,None,2021-03-01,nonPeerReviewed,Article,NaN,NaN,2025-02-28
4,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,None,2023-06-23,nonPeerReviewed,Doctoral thesis,NaN,NaN,2025-02-28
...,...,...,...,...,...,...,...,...
442406,portalunbbra::1a507fabf255873c7efa9a67b3d081a5,None,2014-12-13,peerReviewed,Article,NaN,NaN,2025-02-28
442407,portalunbbra::d22e22c3f5f77ddd0f037c0a1da23a9b,None,2013-12-17,nonPeerReviewed,Article,NaN,NaN,2025-02-28
442408,portalunbbra::fe22079ac12e4b48e955751af4ce027b,CC BY NC ND,2019-04-30,nonPeerReviewed,Article,NaN,NaN,2025-02-28
442409,revistasumft::106b3b4f6115356de22e7aaf07960b34,None,2023-12-31,nonPeerReviewed,Article,NaN,NaN,2025-02-28


In [30]:
df_researchproduct2instance.explode('articleProcessingCharge.amount')

,id,license,publicationDate,refereed,type,articleProcessingCharge.amount,articleProcessingCharge.currency,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,CC BY,2023-12-18,nonPeerReviewed,Article,NaN,NaN,2025-02-28
1,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,None,2014-01-01,peerReviewed,Conference object,NaN,NaN,2025-02-28
2,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,None,2013-01-01,peerReviewed,Conference object,NaN,NaN,2025-02-28
3,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,None,2021-03-01,nonPeerReviewed,Article,NaN,NaN,2025-02-28
4,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,None,2023-06-23,nonPeerReviewed,Doctoral thesis,NaN,NaN,2025-02-28
...,...,...,...,...,...,...,...,...
442406,portalunbbra::1a507fabf255873c7efa9a67b3d081a5,None,2014-12-13,peerReviewed,Article,NaN,NaN,2025-02-28
442407,portalunbbra::d22e22c3f5f77ddd0f037c0a1da23a9b,None,2013-12-17,nonPeerReviewed,Article,NaN,NaN,2025-02-28
442408,portalunbbra::fe22079ac12e4b48e955751af4ce027b,CC BY NC ND,2019-04-30,nonPeerReviewed,Article,NaN,NaN,2025-02-28
442409,revistasumft::106b3b4f6115356de22e7aaf07960b34,None,2023-12-31,nonPeerReviewed,Article,NaN,NaN,2025-02-28


In [ ]:
df_researchproduct2url

,id,url,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,https://revistasinvestigacion.unmsm.edu.pe/ind...,2025-02-28
1,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,http://hdl.handle.net/10261/97737,2025-02-28
2,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,http://hdl.handle.net/10261/97737,2025-02-28
3,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,https://nottingham-repository.worktribe.com/fi...,2025-02-28
4,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,http://sedici.unlp.edu.ar/handle/10915/154923,2025-02-28
...,...,...,...
442405,portalunbbra::0f754e57211d3129ce3271917bfdf22a,http://periodicos.unb.br/index.php/Musica/arti...,2025-02-28
442406,portalunbbra::1a507fabf255873c7efa9a67b3d081a5,http://periodicos.unb.br/index.php/Musica/arti...,2025-02-28
442407,portalunbbra::d22e22c3f5f77ddd0f037c0a1da23a9b,http://periodicos.unb.br/index.php/revistadoce...,2025-02-28
442408,portalunbbra::fe22079ac12e4b48e955751af4ce027b,http://periodicos.unb.br/index.php/belasinfiei...,2025-02-28


In [ ]:
df_researchproduct2alternateIdentifier

,id,scheme,value
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,doi,10.15381/rivep.v34i6.25240
105,dedup_wf_002::003aecb01302b4b4a548b3a8a5f3e78a,doi,10.18566/escr.v31n66.a04
314,dedup_wf_002::00accb67018dda2e1bc0ed6e4ad2306c,doi,10.24215/16696581e180
326,dedup_wf_002::00adb0f98be4e5a699d9737efc5744ae,urn,http://hdl.handle.net/2434/346882
327,dedup_wf_002::00adb0f98be4e5a699d9737efc5744ae,urn,WOS:000361562900076
...,...,...,...
463364,pmid_dedup__::f743342566c26a3dbe272481cbbc4a19,doi,10.1590/s0325-75412010000100003
463373,pmid_dedup__::fa110c5c0c4ff04bd93099a44756e7a4,doi,10.14670/hh-21.849
463379,pmid_dedup__::fb9f0e6468c24d81dc28ad7f39f65d7a,doi,10.1023/a:1006997830346
463382,portalunbbra::062410162331b91ef055c3c5faeb05b7,doi,10.26512/belasinfieis.v8.n2.2019.24383
